In [ ]:
import pickle
import pandas as pd
import numpy as np
import re

%matplotlib inline

## Merge all data from men (n=20,000) and women (n=18,000)

In [ ]:
f_men = open('marathon df_0-20000_2016_men', 'rb')   
df_men = pickle.load(f_men)  
f_men.close() 

f_women = open('marathon df_0-18000_2016_women', 'rb')   
df_women = pickle.load(f_women)  
f_women.close() 

In [ ]:
# Add sex to both
df_men['sex'] = 'male'
df_women['sex'] = 'female'

In [ ]:
df = df_men.append(df_women, ignore_index = True)
df.count()

In [ ]:
df.head()

# Data management  
Several new variables created:  
* Convert final time into minutes  
* Calculate several measures of variation across 5K intervals  
* Convert bib number to integer  
* Convert age groups into ordinal variable  
* Create dummy variables for sex and country

In [ ]:
# Convert final time and interval times into minutes
# In the process, delete observations where extra columns threw data off
df['temp'] = df.finaltime.map(lambda x: x.split(':'))
df['len'] = df.temp.map(lambda x: len(x))
df = df[df['len'] == 3]

df['time'] = df.temp.map(lambda x: int(x[0])*60 + int(x[1]) + int(x[2])/60)
del df['temp']
del df['len']

for i in ['5K', '10K', '15K', '20K', '25K', '30K', '35K', '40K']:
    df[i] = df[i].map(lambda x: x/60.0)

In [ ]:
# How pace changed over time and best time
df['mintime'] = df.loc[:, ['5K', '10K', '15K', '20K', '25K', '30K', '35K', '40K']].min(axis=1)
df['mintime_start'] = df.loc[:, ['5K', '10K', '15K', '20K']].min(axis=1)
df['maxtime'] = df.loc[:, ['5K', '10K', '15K', '20K', '25K', '30K', '35K', '40K']].max(axis=1)
df['timerange'] = df.maxtime - df.mintime

df['start_finish_range'] = (df['5K'] - df['40K']).astype(int)
df['start_finish_range_percent'] = ((df['5K'] - df['40K'])*100/df['5K']).astype(float)

df['start_range'] = (df['5K'] - df['20K']).astype(int)
df['start_range_percent'] = ((df['5K'] - df['20K'])*100/df['5K']).astype(float)

df['finish_range'] = (df['25K'] - df['40K']).astype(int)
df['finish_range_percent'] = ((df['25K'] - df['40K'])*100/df['25K']).astype(float)

In [ ]:
# When mintime occurred
df['peak_time'] = df[['5K', '10K', '15K', '20K', '25K', '30K', '35K', '40K']].idxmin(axis=1)
df['peak_time'] = df.peak_time.map(lambda x: int(x.strip('K')))

df['peak_time_start'] = df[['5K', '10K', '15K']].idxmin(axis=1)
df['peak_time_start'] = df.peak_time_start.map(lambda x: int(x.strip('K')))

In [ ]:
# Calculate SD across intervals and log transform
df['split_std'] = df[['5K', '10K', '15K', '20K', '25K', '30K', '35K', '40K']].std(axis=1)
df['split_std_start'] = df[['5K', '10K', '15K', '20K']].std(axis=1)

df['log_std'] = df.split_std.map(lambda x: np.log(x+1))
df['log_std_start'] = df.split_std_start.map(lambda x: np.log(x+1))

In [ ]:
# Calculate difference from one 5K to next and sum them - overall and first 4 intervals only
for i in range(10, 45, 5):
    name = 'd' + str(i) + 'K'
    df[name] = df[str(i) + 'K'] - df[str(i - 5) + 'K']
    df['d' + str(i) + 'K_abs'] = df['d' + str(i) + 'K'].map(lambda x: abs(x))

df['delta_sums'] = df[['d10K_abs', 'd15K_abs', 'd20K_abs', 'd25K_abs', 
                       'd30K_abs', 'd35K_abs', 'd40K_abs']].sum(axis=1)

df['delta_sums_start'] = df[['d10K_abs', 'd15K_abs', 'd20K_abs', 'd25K_abs']].sum(axis=1)
df['pseudo_gini_start'] = df['delta_sums_start']/df[['5K', '10K', '15K', '20K']].mean(axis=1)

for i in range(10, 45, 5):
    del df['d' + str(i) + 'K_abs']
    
df['log_delta_sums'] = df.delta_sums.map(lambda x: np.log(x+1))
df['log_delta_sums_start'] = df.delta_sums_start.map(lambda x: np.log(x+1))
df['log_gini_start'] = df.pseudo_gini_start.map(lambda x: np.log(x+1))

In [ ]:
# Convert bib number to integer
df['bib'] = df.bib.map(lambda x: int(x))

In [ ]:
# Convert age into an ordinal variable
# In the process, collapse some of the youngest and oldest age groups with sparse data
age_nums = {"age_ord": {"16-19": 1, "20-24": 1, "25-29": 2, "30-34": 3,
                        "35-39": 4, "40-44": 5, "45-49": 6, "50-54": 7,
                        "55-59": 8, "60-64": 9, "65-69": 9, "70-74": 9,
                        "75-79": 9, "80+": 9}}
df['age_ord'] = df.age_group
df.replace(age_nums, inplace=True)

In [ ]:
# Drop 1 observation who is mis-coded
df = df[df.age_ord != 'Group']
df['age_ord'] = df.age_ord.map(lambda x: float(x))

In [ ]:
# Create dummies for gender
df_sex = pd.get_dummies(df['sex'])
df = pd.concat([df, df_sex], axis=1)

In [ ]:
# Create dummies for country
# First flag and drop suspicious ones
df['country'] = df.country.map(lambda x: x.replace('(', '').replace(')', ''))
df['flag'] = df.country.map(lambda x: re.search(r'\b[A-Z]+\b', x))
df = df[df.flag.map(lambda x: bool(x))]
del df['flag']

countries = pd.get_dummies(df['country'])
df = pd.concat([df, countries], axis=1)

In [ ]:
df.head()

In [ ]:
file = 'marathon df_2016_analysis'
fileobj = open(file,'wb') 
pickle.dump(df,fileobj) 